# How do acquisitions change over the next 3 months?

- local time of acquisition
- incidence angle

In [ ]:
import geopandas as gpd
import requests
import json
import csv
import os
import datetime
import pandas as pd

import xyzservices

# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

In [ ]:
with open('/home/jovyan/capella/credentials.json') as f: # USER: Input path to credentials.json file.
    data = json.load(f)
    username = data['username']
    password = data['password']

In [ ]:
# Get a valid token from the auth service
r = requests.post("https://api.capellaspace.com/token", 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))
access_token = r.json()["accessToken"]
# p(accessToken)

# GET user ID and org ID
headers = {'Authorization':'Bearer ' + access_token}
r = requests.get("https://api.capellaspace.com/user", headers=headers)
user_id = r.json()["id"]
org_id = r.json()["organizationId"]
#p(r.json())

# Print user and org ID
print('User email: ', r.json()['email'], '\nOrganization: ', r.json()['organization']['name'], '\nEnvironment: ', r.json()['apiEnvironmentRole'])

In [ ]:
# NOTE: use 3D coordinates for rugged terrain / volcanoes
point = {'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Point',
    'coordinates': (-121.744564, 46.796787, 1589.0)}}]}

In [ ]:
aoi_geom = point['features'][0]['geometry']
aoi_geom

In [ ]:
# Name and description
request_name = 'Rainier - Paradise - Spot - repeat daily' # USER: Set the name of the request.
request_description = 'Mt. Rainier Paradise repeatdaily' # USER: Set a description for the request.

#NOTES
orbital_plane = [53] # options: 45, 53, 97 # only 97 will have polar coverage
orbit_state = "either" # options: ascending, descending, either
look_direction = "either" # options: left, right, either
look_angle_min = 25 # full: 5, extended: 15, standard: 25
look_angle_max = 50 # max: 50

# Footprint size guidelines:
# - Spotlight: length = 5000, width = 5000
# - Sliding Spotlight: length = 10000, width = 5000
# - Stripmap: length = 20000, width = 5000-10000

# STRIP
img_width = 5000 # in meters
img_length = 20000 # in meters

# SLIDE
#img_width = 5000 # in meters
#img_length = 10000 # in meters

# SPOT
#img_width = 5000 # in meters
#img_length = 5000 # in meters

In [ ]:
def submit_request(start_date=datetime.datetime.utcnow(), end_date=None):
    ''' get access estimated for start+30 days'''
    # Access request window
    # Default: 30-day window starting at current UTC date/time
    # MAX 90-days out
    ts90 = pd.Timestamp.today('UTC') + pd.Timedelta('89D')

    if end_date == None:
        end_date = start_date + datetime.timedelta(days = 30) # 31 days or less!!!
    elif end_date >= ts90:
        print('setting max window close (today + 90 days)')
        end_date = ts90

    # USER: To change the window from the default, uncomment and set the start_date and end_date for your desired window in UTC in YYYY,M,D format.
    # start_date = datetime.date(YYYY,M,D)
    # end_date = datetime.date(YYYY,M,D)
    window_open = start_date.strftime("%Y-%m-%dT%H:%M:%SZ") # format start date
    window_close = end_date.strftime("%Y-%m-%dT%H:%M:%SZ") # format end date
    print(f'Searching {window_open} To {window_close}')
    
    # Define access request
    access_request = {
        "geometry": aoi_geom,
        "properties": {
          "userId": user_id,
          "orgId": org_id,
          "accessrequestDescription": request_description,
          "accessrequestName": request_name,
          "windowOpen": window_open,
          "windowClose": window_close,
          "accessConstraints": {
              "ascDsc": orbit_state,
              "lookDirection": look_direction,
              "orbitalPlanes": orbital_plane,
              "offNadirMin": look_angle_min,
              "offNadirMax": look_angle_max, #offNadirMax #seems possibly stuck on 40?
              "imageWidth": img_width,
              "imageLength": img_length
          }
        },
        "type": "Feature"
    }

    # POST request
    headers = {'Authorization':'Bearer ' + access_token}
    r = requests.post("https://api.capellaspace.com/ma/accessrequests/", json = access_request, headers = headers)
    #p(r.json())
    return r

In [ ]:
r1 = submit_request()

In [ ]:
# Results retrieved via access request ID:
ar_id = r1.json()['properties']['accessrequestId']
ar_id

In [ ]:
# Check after a bit
status = requests.get("https://api.capellaspace.com/ma/accessrequests/" + ar_id, headers = headers)
status.json() # See processing: 'completed'

In [ ]:
# GET access requests
results = requests.get("https://api.capellaspace.com/ma/accesses/" + ar_id, headers=headers)

# View requests as table
df1 = pd.json_normalize(results.json())
col_interest = ['spacecraftId', 'windowOpen', 'windowClose', 'orbitalPlane', 'accessProperties.ascdsc', 
                'accessProperties.lookDirection', 'accessProperties.offNadirMin']

#access_df # Uncomment to view full table of access requests
print('Number of possible acquisitions:', len(df1))
df1[col_interest].head(10) # Uncomment to view access requests only with columns of interest

In [ ]:
tmp = df.sort_values('windowOpen')#.reset_index(drop=True)
print(tmp.windowOpen.iloc[0])
print(tmp.windowOpen.iloc[-1])
tmp.to_csv('access_30.csv', index=False) #unnamed??
#tmp.tail()

In [ ]:
df = pd.read_csv('access_30.csv')
df.windowOpen.iloc[-1] #windowClose': '2023-12-11T05:29:26.000Z', NOTE access request

In [ ]:
import time

In [ ]:
# Another request 30 days from last date
start = pd.to_datetime(df1.windowOpen.iloc[-1]) + datetime.timedelta(hours = 1)
r = submit_request(start_date=start)
time.sleep(5) #takes a little time to post
# Results retrieved via access request ID:
ar_id2 = r.json()['properties']['accessrequestId']
print(ar_id2)
results = requests.get("https://api.capellaspace.com/ma/accesses/" + ar_id2, headers=headers)

# View requests as table
df2 = pd.json_normalize(results.json())
df2.head()

In [ ]:
len(df2)

In [ ]:
tmp = df2.sort_values('windowOpen')#.reset_index(drop=True)
print(tmp.windowOpen.iloc[0])
print(tmp.windowOpen.iloc[-1])

In [ ]:
tmp.to_csv('access_60.csv', index=False)

In [ ]:
# Another request 30 days from last date
start = pd.to_datetime(df2.windowOpen.iloc[-1]) + datetime.timedelta(hours = 1)
r = submit_request(start_date=start)
time.sleep(5) #takes a little time to post
# Results retrieved via access request ID:
ar_id3 = r.json()['properties']['accessrequestId']
print(ar_id3)
results = requests.get("https://api.capellaspace.com/ma/accesses/" + ar_id3, headers=headers)

# View requests as table
df3 = pd.json_normalize(results.json())
df3.head()

In [ ]:
len(df3)

In [ ]:
tmp = df3.sort_values('windowOpen')#.reset_index(drop=True)
print(tmp.windowOpen.iloc[0])
print(tmp.windowOpen.iloc[-1])

In [ ]:
tmp.to_csv('access_90.csv', index=False)

In [ ]:
pd.to_datetime(tmp.windowOpen.iloc[-1]) - pd.Timestamp.today('UTC')

In [ ]:
# Should fail b/c 90 days past current date? "windowClose must be within 90 days of the current date
# {'error': {'message': "Cannot read property 'accessrequestId' of undefined"}} had to re-login
# NOTE: likely happening b/c no acquisitions that week!

start = pd.to_datetime(df3.windowOpen.iloc[-1]) + datetime.timedelta(hours = 1)
#end = pd.Timestamp.today('UTC') + pd.Timedelta('89D')
end = start + datetime.timedelta(days = 6)

r = submit_request(start_date=start, end_date=end)
#time.sleep(10) #takes a little time to post

In [ ]:
r.json()

In [ ]:
# Results retrieved via access request ID:
ar_id4 = r.json()['properties']['accessrequestId']
print(ar_id4)
results = requests.get("https://api.capellaspace.com/ma/accesses/" + ar_id4, headers=headers)

In [ ]:
results.json()

In [ ]:
# View requests as table
df4 = pd.json_normalize(results.json())
df4.head()

In [ ]:
r.json()

In [ ]:
# No Capella-11 Acadia?
access_df.spacecraftId.unique()

In [ ]:
access_df.tileId.unique()

In [ ]:
r = requests.get("https://api.capellaspace.com/tiles/list/accessrequest/" + ar_id, headers = headers)

In [ ]:
gff = gpd.GeoDataFrame.from_features(r.json(), crs='EPSG:4326')
gff.explore()

In [ ]:
# For some reason, accessrequest tiles do no constrain by orbit...
gff[gff.tileId.isin(access_df.tileId.unique())].explore()

In [ ]:
access_df_summary = access_df.groupby(['orbitalPlane',
            'accessProperties.ascdsc', 
            'accessProperties.lookDirection'
           ]).agg(passCount=('windowOpen', 'count')
                 #  LookDirAzimuth=('LookDirectionAzimuth[degrees]', 'mean'),
                 # LookDirAzimuthStd=('LookDirectionAzimuth[degrees]', 'std'),
                 #  SatAzimuth = ('SatelliteAzimuth[degrees]', 'mean')
                 ).reset_index().rename(columns = {'accessProperties.ascdsc': 'orbitState', 'accessProperties.lookDirection': 'lookDirection'})
access_df_summary

In [ ]:
#access_df.sort_values(column='windowOpen')
access_df.iloc[0]

In [ ]:
# # For a given combination of orbitState and lookDirection, how do acquisitions vary over time?
# Use STAC Names for anaylsis
df = access_df.reset_index().rename(columns = {'accessProperties.ascdsc': 'orbit_state', 
                                               'accessProperties.lookDirection': 'observation_direction',
                                               'accessProperties.offNadirMin': 'look_angle'}) # Match console
tmp = df[(df.orbit_state == 'ascending') & (df.observation_direction == 'right')]

In [ ]:
# Not sure what is going on with azimuthClose
#summarize = ['accessProperties.azimuthOpen', 'accessProperties.azimuthClose']
#tmp[summarize].describe()
tmp

In [ ]:
tmp['windowOpenLocal'] = pd.to_datetime(tmp.windowOpenLocal)
tmp.plot(x='windowOpenLocal', y='look_angle', marker='o')
